# Petit programme pour faire des groupes d'élèves
*Mélanie - Marc-André - décembre 2017 / janvier 2018*

## Initialisation

In [1]:
import collections
import numpy as np
from collections import namedtuple
from random import sample, randrange, random
import csv
import copy
with open('list_6e5.csv', newline='') as csvfile:
    persons = csv.reader(csvfile, delimiter=',', quotechar='|')
    pers = list(persons)   


## Structure de départ en listes puis dictionnaires

In [2]:
# V.3 liste
def creategroupes(liste_p, nb_min_pers):
    """
    renvoie une liste de listes (groupes d'élèves) 
    à partir d'une liste d'élèves non-groupés
    en fonction du nombre minimum d'élèves par groupe
    """
    nb_grp, r = divmod(len(liste_p), nb_min_pers)
    groupes = [liste_p[p:p+nb_min_pers] for p in range(0, nb_grp*nb_min_pers, nb_min_pers)]
    for p in range(nb_grp*nb_min_pers, len(liste_p)):  # ensuite ceux qui restent
        groupes[p%nb_grp].append(liste_p[p])
    return groupes

In [3]:
def lx(liste_p):
    """
    renvoie une liste de listes des différents groupes d'élèves qui ne doivent
    pas être dans le même groupe
    """
    from collections import defaultdict
    dicex = defaultdict(list)
    for e in liste_p:
        dicex[e[3]].append(e[:2])
    return [e for k, e in dicex.items() if k is not ""]   

In [4]:
exclus = lx(pers)
exclus

[[['BERNARD Olivier', '1'],
  ['KIESELE Killian', '1'],
  ['SIMSEK Kenan', '2'],
  ['TUUGAHALA Teiva', '2']]]

In [5]:
groups = creategroupes(pers, 3)
groups

[[['BERNARD Olivier', '1', 'm', 'g1'],
  ['BESLAY Antoine', '4', 'm', ''],
  ['BINA Juliette', '2', 'f', ''],
  ['TURAN Aysu', '3', 'f', '']],
 [['BOUSILA Benjamin', '4', 'm', ''],
  ['CHABAN Sonia', '5', 'f', ''],
  ['DA SILVA Line', '2', 'f', ''],
  ['TUUGAHALA Teiva', '2', 'm', 'g1']],
 [['GALJAARDT Fleur', '4', 'f', ''],
  ['JACQUIN Manon', '5', 'f', ''],
  ['KIESELE Killian', '1', 'm', 'g1']],
 [['PEIFFER Jules', '4', 'm', ''],
  ['SEMBACH Childéric', '5', 'm', ''],
  ['SIMSEK Kenan', '2', 'm', 'g1']]]

## Définition et calcul des contraintes

Plus les contraintes sont respectées, plus le retour chiffré est proche de zéro.


In [6]:
# contraintes
def unbon(group):   
    "il faut au moins un bon"
    
    niveaux = [int(g[1]) for g in group] 
    result = (4-(max(niveaux)-min(niveaux))) + (1/np.std(niveaux))
       
    return result

def mixmf(group):
    "il faut mélanger m et f"
    m = 0
    f = 0
    for g in group:
        if g[2] == 'm':
            m += 1
        else:
            f += 1
    return abs(m-f)

def exclusions(group, exclus = exclus):
    """
    Elle empêche que des élèves exclus se retrouvent ensemble
    Elle récupère le nombre d'élèves exclus présents dans le 
    groupe et donne une note en fonction
    """
    result = 0
    elevesX = [(g[0]) for g in group if g[0] in exclus]
    if len(elevesX) > 1:
        result = len(elevesX)-1
    return result                           

In [7]:
# V. liste
def calc_score(groups, exclus):
    """
    renvoie le score de la classe en fonction des
    résultats obtenus pour chaque critère dans chaque groupe
    permet de pondérer les différents critères
    """
    score = 0
    for group in groups:  
        score += 3*unbon(group)
        score += mixmf(group)
        score += exclusions(group, exclus)
    return score

In [8]:
score_classe = calc_score(groups, exclus)
score_classe

22.163079043213727

## Inversion 

Permutation au hasard de deux élèves de deux groupes différents

In [9]:
def deplace(config_avant):
    """
    choisir au hasard deux élèves dans deux groupes différents
    les échanger de groupe
    """
    from copy import deepcopy
    # prend deux groupes au hasard
    g1, g2 = sample(range(len(config_avant)), 2)
    # prend un élève au hasard dans chacun
    e1 = randrange(len(config_avant[g1]))
    e2 = randrange(len(config_avant[g2]))
    # tu les échanges
    config_apres = deepcopy(config_avant)
    config_apres[g2][e2], config_apres[g1][e1] = config_avant[g1][e1], config_avant[g2][e2]
    return config_apres

In [10]:
group_modif = deplace(groups)
group_modif, calc_score(group_modif, exclus)

([[['BERNARD Olivier', '1', 'm', 'g1'],
   ['BESLAY Antoine', '4', 'm', ''],
   ['BINA Juliette', '2', 'f', ''],
   ['TURAN Aysu', '3', 'f', '']],
  [['BOUSILA Benjamin', '4', 'm', ''],
   ['CHABAN Sonia', '5', 'f', ''],
   ['DA SILVA Line', '2', 'f', ''],
   ['TUUGAHALA Teiva', '2', 'm', 'g1']],
  [['SEMBACH Childéric', '5', 'm', ''],
   ['JACQUIN Manon', '5', 'f', ''],
   ['KIESELE Killian', '1', 'm', 'g1']],
  [['PEIFFER Jules', '4', 'm', ''],
   ['GALJAARDT Fleur', '4', 'f', ''],
   ['SIMSEK Kenan', '2', 'm', 'g1']]],
 23.765653422767446)

## Composition du meilleur groupe

Vérification de la pertinence du changement jusqu'à obtention du score le plus bas possible

In [11]:
def algo1(eleves, taillegroupe, exclus, grand_nombre=1000):
    """
    vérifie que la configuration après échange donne un meilleur score, 
    sinon annule et teste un autre échange,
    le tout un très grand nombre de fois pour s'approcher
    du meilleur score possible
    """
    
    # genre... une recherche au hasard
    config = creategroupes(eleves, taillegroupe)
    meilleure = config                   # on garde la meilleure "so-far"
    score_meil = calc_score(meilleure, exclus)   # la première fait l'affaire.
    for i in range(grand_nombre):
        score = calc_score(config, exclus)
        config_apres = deplace(config)
        score_apres = calc_score(config_apres, exclus)
        if score_apres<score_meil:       # garde si c'est meilleur
            score_meil = score_apres
            meilleure = config
        config = config_apres # et déplace - toujours - marche au hasard !
    return meilleure

In [12]:
def metropolis(init, exclus, temperature=10, iterations=1000):
    """
    vérifie que la configuration après échange donne un meilleur score, 
    sinon annule et teste un autre échange,
    le tout un très grand nombre de fois pour s'approcher
    du meilleur score possible
    """
    
    # genre... une recherche au hasard
    config = init
    meilleure = config                   # on garde la meilleure "so-far"
    score_meil = calc_score(meilleure, exclus)   # la première fait l'affaire.
    for i in range(iterations):
        score = calc_score(config, exclus)
        config_apres = deplace(config)
        score_apres = calc_score(config_apres, exclus)
        if score_apres<score_meil:       # garde si c'est meilleur
            score_meil = score_apres
            meilleure = config
        if score>score_apres:
            config = config_apres # et déplace - toujours - marche au hasard !
        else:
            if random() < 0.5:
                config = config_apres
    return meilleure

In [13]:
config = creategroupes(pers,3)


In [14]:
result = metropolis(config, exclus, iterations = 1000)
result, calc_score(result, exclus)

/home/mel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  


([[['SEMBACH Childéric', '5', 'm', ''],
   ['DA SILVA Line', '2', 'f', ''],
   ['TUUGAHALA Teiva', '2', 'm', 'g1'],
   ['TURAN Aysu', '3', 'f', '']],
  [['BERNARD Olivier', '1', 'm', 'g1'],
   ['BINA Juliette', '2', 'f', ''],
   ['CHABAN Sonia', '5', 'f', ''],
   ['BESLAY Antoine', '4', 'm', '']],
  [['BOUSILA Benjamin', '4', 'm', ''],
   ['GALJAARDT Fleur', '4', 'f', ''],
   ['SIMSEK Kenan', '2', 'm', 'g1']],
  [['PEIFFER Jules', '4', 'm', ''],
   ['JACQUIN Manon', '5', 'f', ''],
   ['KIESELE Killian', '1', 'm', 'g1']]],
 20.293882070467326)

In [15]:
def montre(groups):
    "montre le résultat"
    for i, g in enumerate(groups):
        print("Groupe %d"%(i+1))
        print("=========")
        for e in g:
            print(e[0], end=", ")
        print()
        for func in (unbon, mixmf, exclusions):
            print("%s : %.2f"%(func.__name__, func(g)))
    print("note totale: %.2f"%calc_score(groups, exclus))

In [16]:
montre(result)

Groupe 1
SEMBACH Childéric, DA SILVA Line, TUUGAHALA Teiva, TURAN Aysu, 
unbon : 1.82
mixmf : 0.00
exclusions : 0.00
Groupe 2
BERNARD Olivier, BINA Juliette, CHABAN Sonia, BESLAY Antoine, 
unbon : 0.63
mixmf : 0.00
exclusions : 0.00
Groupe 3
BOUSILA Benjamin, GALJAARDT Fleur, SIMSEK Kenan, 
unbon : 3.06
mixmf : 1.00
exclusions : 0.00
Groupe 4
PEIFFER Jules, JACQUIN Manon, KIESELE Killian, 
unbon : 0.59
mixmf : 1.00
exclusions : 0.00
note totale: 20.29


In [17]:
from IPython.display import HTML, display_html
from datetime import datetime

In [18]:
def montre_2(groups):
    score = calc_score(groups, exclus)            
    dd = "<table border=1><tr><th colspan={} style ='background-color:#cdcdf0;'><div style='text-align:center;font-size:30px;'> Groupes de la classe</div><div style='text-align:right;'> {} --------> score : {sc:.2f} </div></th></tr>".format(len(groups)+1, datetime.now().strftime('%Y-%m-%d'), sc=score)
    for i,j in enumerate(groups):
        dd += "<tr><td style='font-weight:bold;background-color:#cdcdf0;'>{}</td>".format("Groupe " + str(i+1))
        for g in j:
            dd += "<td style='text-align:left;'>" + g[0] + "</td>"
        dd += "</tr>"
    dd += "</table>"
        
    display_html(HTML(dd))
        
    

In [19]:
montre_2(result)

Groupes de la classe 2019-11-16 --------> score : 20.29 Groupe 1 SEMBACH Childéric DA SILVA Line TUUGAHALA Teiva TURAN Aysu Groupe 2 BERNARD Olivier BINA Juliette CHABAN Sonia BESLAY Antoine Groupe 3 BOUSILA Benjamin GALJAARDT Fleur SIMSEK Kenan Groupe 4 PEIFFER Jules JACQUIN Manon KIESELE Killian

In [20]:
# là, je sépare simplement le html et le python en suivant ton code
temptable = "<table border=1><tr><th colspan="+str(len(groups)+1)+" style ='background-color:#cdcdf0;'><div style='text-align:center;font-size:30px;'> Groupes de la classe</div><div style='text-align:right;'>" + datetime.now().strftime('%Y-%m-%d') + "--------> score : {} </div></th></tr>{}</table>"
templine = "<tr><td style='font-weight:bold;background-color:#cdcdf0;'>Groupe {}</td>{}</tr>"
tempcell = "<td style='text-align:left;'>{}</td>"
def montre_3(groups):
    score = calc_score(groups, exclus)
    lines = ""
    for i,j in enumerate(groups):
        cells = ""
        for g in j:
            cells += tempcell.format(g[0])
        lines += templine.format(i+1, cells)
    table = temptable.format(round(score,2), lines)
    display_html(HTML(table))
        
montre_3(result)

Groupes de la classe 2019-11-16--------> score : 20.29 Groupe 1 SEMBACH Childéric DA SILVA Line TUUGAHALA Teiva TURAN Aysu Groupe 2 BERNARD Olivier BINA Juliette CHABAN Sonia BESLAY Antoine Groupe 3 BOUSILA Benjamin GALJAARDT Fleur SIMSEK Kenan Groupe 4 PEIFFER Jules JACQUIN Manon KIESELE Killian

In [21]:
# la même chose, mais sans la boucle centrale (j'aime bien - je trouve plus lisible)
temptable = "<table border=1><tr><th colspan="+str(len(groups)+1)+" style ='background-color:#cdcdf0;'><div style='text-align:center;font-size:30px;'> Groupes de la classe</div><div style='text-align:right;'>" + datetime.now().strftime('%Y-%m-%d') + "--------> score : {} </div></th></tr>{}</table>"
templine = "<tr><td style='font-weight:bold;background-color:#cdcdf0;'>Groupe {}</td>{}</tr>"
tempcell = "<td style='text-align:left;'>{}</td>"
def montre_4(groups):
    score = calc_score(groups, exclus)
    lines = ""
    for i,j in enumerate(groups):
        cells = " ".join([tempcell.format(g[0]) for g in j])
        lines += templine.format(i+1, cells)
    table = temptable.format(round(score,2), lines)
    display_html(HTML(table))
        
montre_4(result)

Groupes de la classe 2019-11-16--------> score : 20.29 Groupe 1 SEMBACH Childéric DA SILVA Line TUUGAHALA Teiva TURAN Aysu Groupe 2 BERNARD Olivier BINA Juliette CHABAN Sonia BESLAY Antoine Groupe 3 BOUSILA Benjamin GALJAARDT Fleur SIMSEK Kenan Groupe 4 PEIFFER Jules JACQUIN Manon KIESELE Killian

In [22]:
# avec un vrai template
temptable = """
<table border=1>
    <tr><th colspan={tablelen} style ='background-color:#cdcdf0;'>
        <div style='text-align:center;font-size:30px;'> Groupes de la classe</div><div style='text-align:right;'>
        {date} --------> score : {score} </div></th></tr>
    {lines}
</table>
"""
templine = "<tr><td style='font-weight:bold;background-color:#cdcdf0;'>Groupe {}</td>{}</tr>"
tempcell = "<td style='text-align:left;'>{}</td>"
def montre_5(groups):
    score = round((calc_score(groups, exclus)),2)
    lines = ""
    for i,j in enumerate(groups):
        cells = " ".join([tempcell.format(g[0]) for g in j])
        lines += templine.format(i+1, cells)
    tablelen = len(groups)+1
    date = datetime.now().strftime('%Y-%m-%d')
    table = temptable.format(tablelen=tablelen, date=date, score=score, lines=lines)
    display_html(HTML(table))
        
montre_5(result)

Groupes de la classe 
 2019-11-16 --------> score : 20.29 
 Groupe 1 SEMBACH Childéric DA SILVA Line TUUGAHALA Teiva TURAN Aysu Groupe 2 BERNARD Olivier BINA Juliette CHABAN Sonia BESLAY Antoine Groupe 3 BOUSILA Benjamin GALJAARDT Fleur SIMSEK Kenan Groupe 4 PEIFFER Jules JACQUIN Manon KIESELE Killian

In [23]:
import jinja2
from jinja2 import Template
temptable = """
<table border=1>
    <tr><th colspan={{tablelen}} style ='background-color:#cdcdf0;'>
        <div style='text-align:center;font-size:30px;'> Groupes de la classe</div><div style='text-align:right;'>
        {{date}} --------> score : {{"%.2f"|format(score)}} </div></th></tr>
    {% for j in groups %}
        <tr>
            <td style='font-weight:bold;background-color:#cdcdf0;'>Groupe {{loop.index}} </td>
            {% for g in j %}
                <td style='text-align:left;'>{{g[0]}}</td>
            {% endfor %}
        </tr>
    {% endfor %}
</table>
"""


    
def montre_6(groups):
    score = calc_score(groups, exclus)    # je calcule tous les champs
    tablelen = len(groups)+1
    date = datetime.now().strftime('%Y-%m-%d')
    table = Template(temptable)           # je crée le prgm template avec le bon html jinja
    html = table.render(locals())         # je fais le rendu, en passant locals()
    display_html(HTML(html))              # c'est du html

montre_6(result)



Groupes de la classe 
 2019-11-16 --------> score : 20.29 
 
 
 Groupe 1 
 
 SEMBACH Childéric 
 
 DA SILVA Line 
 
 TUUGAHALA Teiva 
 
 TURAN Aysu 
 
 
 
 
 Groupe 2 
 
 BERNARD Olivier 
 
 BINA Juliette 
 
 CHABAN Sonia 
 
 BESLAY Antoine 
 
 
 
 
 Groupe 3 
 
 BOUSILA Benjamin 
 
 GALJAARDT Fleur 
 
 SIMSEK Kenan 
 
 
 
 
 Groupe 4 
 
 PEIFFER Jules 
 
 JACQUIN Manon 
 
 KIESELE Killian